# Projeção de ocupação de leitos enfermaria e UTI em estados com modelos SEIR e SEAPMDR

Originalmente, a ferramenta **SimulaCovid** utiliza um modelo epidemiológico SEIR (Suscetíveis, Expostos, Infectados e Removidos) para calcular a disseminação e evolução clínica do COVID-19. Esse modelo original era baseado na implementação desenvolvida por [Hill (2020)](https://github.com/alsnhll/SEIR_COVID19) e dividia a população em diferentes "compartimentos"
mutuamente excludentes de indivíduos:

- Suscetíveis ($S$): aqueles que não têm imunidade à doença, estes se tornam
expostos à doença a uma taxa $\\beta_i$ devido ao contato com indivíduos
infetados $I_i$;
- Expostos ($E$): aqueles que entram em contato com a doença e desenvolvem
ou não sintomas, mas ainda não transmitem infecção. Esses indivíduos
expostos progridem para o primeiro estágio de infecção ($I_1$), leve, a
uma taxa $\sigma$.
- Infectados ($I$): são aqueles que desenvolvem sintomas e transmitem a
doença a uma taxa $\beta_i$. Os casos de infecção são separados em
diferentes estágios de gravidade:
- Leve ($I_1$): aquele que não necessita de hospitalização, progride para
    o estado severo a uma taxa $p_1$, ou se recupera a uma taxa $\gamma_1$;
- Severo ($I_2$): aquele que necessita de hospitalização, progride para o
    estado crítico a uma taxa $p_1$, ou se recupera a uma taxa $\gamma_2$;
- Grave ($I_3$): aquele que além de hospitalização, necessita de
    tratamento intensivo com equipamento de ventiladores. Chegando ao caso
    grave, o indivíduo pode se recuperar a uma taxa $\gamma_3$ ou o quadro
    pode levar à morte com uma taxa $\mu$;
- Recuperados ($R$): aqueles que, após o curso da doença, se recuperam e
desenvolvem imunidade - não retornam ao estado suscetível.
- Mortos ($D$): indivíduos que morrem pelo agravamento da infecção.

Conforme a descrição acima, o conjunto de equações que determinavam a dinâmica do modelo era dado por:

$\frac{dS}{dt} = - (\beta_1 I_1 + \beta_2 I_2 + \beta_3 I_3) S$

$\frac{dE}{dt} = (\beta_1 I_1 + \beta_2 I_2 + \beta_3 I_3) S - \sigma E$

$\frac{dI_1}{dt} = \sigma E - (γ_1 + p_1) I_1$

$\frac{dI_2}{dt} = p_1 I_1 - (γ_2 + p_2) I_2$

$\frac{dI_3}{dt} = p_2 I_2 - (γ_3 + μ) I_3$

$\frac{dR}{dt} = γ_1 I_1 + γ_2 I_2 + γ_3 I_3$

$\frac{dD}{dt} = μ I_3$


### Atualizações do modelo

Esse modelo, no entanto, ignora algumas características importantes da transmissão da Covid-19, que foram se tornando mais claras com a evolução do conhecimento sobre a doença. 

Em especial, diversos trabalhos (por exemplo, [Tindale et al., 2020]; [Lee et
  al., 2020]; e [Casey et al., 2020]) têm destacado a existência de transmissão secundária por pacientes pré-sintomáticos (expostos, mas que ainda não desenvolveram sintomas) e assintomáticos (infectados que jamais desenvolverão sintomas). Do ponto de vista da saúde pública, a ausência desses grupos na modelagem epidemiológica pode levar a uma subestimativa a taxa de transmissão no início de um surto local, e a uma superestimativa no número de mortes ([Yang, Lombardi Jr. e Yang, 2020b]).

Para mitigar essa possibilidade, implementamos um modelo mais representativo da história natural da enfermidade. Assim como o modelo original, esse novo modelo foi baseado na implementação proposta por [Hill (2020)] - que, desde março de 2020, também foi alterada para comportar grupos de indivíduos assintomáticos e pré-sintomáticos capazes de transmitir a doença. Com esses novos compartimentos, a implementação se aproxima mais de um modelo SEAPMDR (Suscetíveis \[S], Expostos \[E], Assintomáticos \[A], Pré-sintomáticos \[P], Moderados [M], Severos \[D] e Recuperados \[R]), conforme proposto por [Yang, Lombardi Jr. e Yang (2020a)].

Com os novos compartimentos, o grupo de Expostos ($E$) original passa a ser dividido em um compartimento $E_0$, de indivíduos expostos com o agente infeccioso latente (não transmissível); e um compartimento $E_1$, composto por indivíduos pré-sintomáticos (e transmissíveis):

- Latentes ($E_0$): aqueles que entraram em contato com a doença mas ainda não desenvolveram a doença nem se tornaram transmissíveis. Progridem para o estágio $E_1$ a uma taxa $\sigma_0$.
- Pré-sintomáticos ($E_0$): aqueles que entraram em contato com a doença, ainda não desenvolveram a doença, mas já transmitem o agente infeccioso a uma taxa $\beta_E$. Progridem para o estágio $I_1$ ou $I_0$ *(ver abaixo)* a uma taxa $\sigma_1$.

Além dos casos pré-sintomáticos, foi considerado um novo subcompartimento de infectados: aqueles que, passado o período de incubação, jamais chegam a desenvolver sintomas - mas continuam sendo transmissíveis durante um certo período, antes de se recuperarem completamente. Esse subcompartimento foi denominado $I_0$, e comporta uma proporção $\phi$ dos indivíduos que entram no compartimento $I$ (Infectados):

- Assintomáticos ($I_0$): aqueles indivíduos que já superaram o período de incubação, mas não chegam a demonstrar sintomas. Ainda assim, transmitem a doença a uma taxa $\beta_0$. Se recuperam completamente (e deixam de ser transmissíveis) a uma taxa $\gamma_0$.

Com essas mudanças, as equações atualizadas do modelo passam a ser:


$\frac{dS}{dt} = - (\beta_E E_1 + \beta_1 I_1 + \beta_2 I_2 + \beta_3 I_3) S$

$\frac{dE_0}{dt} = (\beta_E E_1 + \beta_1 I_1 + \beta_2 I_2 + \beta_3 I_3) S - \sigma_0 E_0$

$\frac{dE_1}{dt} = \sigma_0 E_0 - \sigma_1 E_1$

$\frac{dI_0}{dt} = \sigma_1 E_1 \phi - \gamma_0 I_0$

$\frac{dI_1}{dt} = \sigma_1 E_1 (1- \phi) - (\gamma_1 + p_1) I_1$

$\frac{dI_2}{dt} = p_1 I_1 - (\gamma_2 + p_2) I_2$

$\frac{dI_3}{dt} = p_2 I_2 - (\gamma_3 + μ) I_3$

$\frac{dR}{dt} = \gamma_1 I_1 + \gamma_2 I_2 + \gamma_3 I_3$

$\frac{dD}{dt} = \mu I_3$

#### Outras alterações

Outra mudança realizada em relação à implementação original foi a utilização da taxa de transmissão efetiva estimada para o local para calcular a proporção de indivíduos suscetíveis ($S$) são expostos à doença (entram para o compartimento $E_0$) a cada intervalo de tempo. 

Anteriormente, esse cálculo era realizado com uma taxa padrão para a doença - o que ignora as trajetórias potencialmente muito diferentes da epidemia em cada local. Ao invés disso, são utilizadas as estimativas já geradas pelo FarolCovid para as taxas de transmissão locais, seguindo a fórmula (disponibilizada por Wicklin, 2020, entre outros):

$D = \ln(2) / R_t$

onde $D$ é o número de dias para que o número de casos dobre; e $R_t$ é a estimava da taxa de transmissão local.

[Casey et al., 2020]: https://doi.org/10.1101/2020.05.08.20094870
[Hill (2020)]: https://github.com/alsnhll/SEIR_COVID19
[Lee et al., 2020]: https://doi.org/10.1007/s12630-020-01729-x
[Tindale et al., 2020]: https://doi.org/10.7554/eLife.57149
[Yang, Lombardi Jr. e Yang (2020a)]: https://arxiv.org/abs/2004.05715
[Yang, Lombardi Jr. e Yang, 2020b]: https://doi.org/10.1101/2020.10.11.20210831


## Importa bibliotecas

In [1]:
from pathlib import Path

import pandas as pd
import requests
import yaml


# HACK: allows sibling directory imports in notebooks
# CREDIT: https://stackoverflow.com/a/60478241
import sys
sys.path.insert(0, '..') # add parent folder path where lib folder is
from simulacovid import prepare, seir, seapmdr, simulator

## Como rodar a simulação para os estados

### Carrega parâmetros fixos do arquivo de configuração

A ferramenta [**FarolCovid**](https://github.com/ImpulsoGov/farolcovid) disponibiliza um arquivo de configuração que inclui dados dos tempos de progressão da doença, preparados para os cálculos do simulador. 

Como a versão SEAPMDR exige alguns parâmetros inexistentes nas configurações originais, foi disponibilizado [no repositório](https://github.com/bcbernardo/impulsogov-desafio-simulacovid/blob/master/custom_configs.yaml) uma versão modificada com as configurações relevantes.

In [2]:
# Get from GitHub
#url = (
#    "https://raw.githubusercontent.com/bcbernardo/impulsogov-desafio-simulacovid/" +
#    "master/custom_configs.yaml"
#)

#config = yaml.load(requests.get(url).text, Loader=yaml.FullLoader)

# OR get local
with open(Path("..", "custom_configs.yaml"), "r") as f:
    config = yaml.load(f, Loader=yaml.FullLoader)

config["br"]["seir_parameters"]

{'asymptomatic_proportion': 0.3,
 'asymptomatic_duration': 6,
 'mild_duration': 6,
 'severe_duration': 6,
 'critical_duration': 8,
 'fatality_ratio': 0.02,
 'doubling_rate': 1.15,
 'incubation_period': 5,
 'presymptomatic_period': 2,
 'i0_percentage': 0.3,
 'i1_percentage': 0.5985,
 'i2_percentage': 0.084,
 'i3_percentage': 0.0175,
 'infected_health_care_proportion': 0.05,
 'hospitalized_by_age_perc': {'from_0_to_9': 2e-05,
  'from_10_to_19': 0.0004,
  'from_20_to_29': 0.011,
  'from_30_to_39': 0.034,
  'from_40_to_49': 0.043,
  'from_50_to_59': 0.082,
  'from_60_to_69': 0.118,
  'from_70_to_79': 0.166,
  'from_80_to_older': 0.184}}

### Guarda a correspondência entre estados e os respectivos códigos do IBGE

Alguns dos conjuntos de dados utilizados identificam as unidades da federação por meio de siglas, enquanto outros utilizam os geocódigos [fornecidos pelo IBGE][API UFs]. Guardamos esta correspondência em um *DataFrame*, para permitir o cruzamento entre os conjuntos de dados mais à frente.

[API UFs]: https://servicodados.ibge.gov.br/api/docs/localidades?versao=1#api-UFs-estadosGet

In [3]:
state_num_ids = pd.DataFrame([  # SEE: https://servicodados.ibge.gov.br/api/v1/localidades/estados/
        ("RO", 11),
        ("AC", 12),
        ("AM", 13),
        ("RR", 14),
        ("PA", 15),
        ("AP", 16),
        ("TO", 17),
        ("MA", 21),
        ("PI", 22),
        ("CE", 23),
        ("RN", 24),
        ("PB", 25),
        ("PE", 26),
        ("AL", 27),
        ("SE", 28),
        ("BA", 29),
        ("MG", 31),
        ("ES", 32),
        ("RJ", 33),
        ("SP", 35),
        ("PR", 41),
        ("SC", 42),
        ("RS", 43),
        ("MS", 50),
        ("MT", 51),
        ("GO", 52),
        ("DF", 53),
    ],
    columns=["state_id", "state_num_id"]
)

### Carrega parâmetros de hospitalização e mortalidade por estado

**Conforme a implementação original**, passamos a utilizar parâmetros calculados por nível geográfico (neste caso estado) levando em consideração a distriuição etária da população. Abaixo segue um trecho de a metodologia com mais detalhes sobre esse cálculo:

> A proporção esperada de hospitalizações é dada pela estimativa de hospitalizações de cada faixa etária, de Verity, Robert, et al. (2020) num amplo estudo com base em 3.665 casos na China, ponderado pelo total da população em cada faixa etária (População residente em 2019 - CNES). Esse valor nos fornece o percentual total de hospitalizações esperadas na população (I2+I3/I). Para obter o percentual por intensidade do caso, mantivemos a razão entre severos (I2) e críticos (I3) constante: antes tínhamos 12.5% severos e 2.5% críticos do total de casos ativos (I) - ou seja, uma razão de 0.2 críticos para cada caso severo (I3/I2) -, que passa a ser de 83.3% severos (I2) e 16.7% críticos (I3) do total estimado de hospitalizados (I2+I3 / I).

Para a incorporação da categoria de assintomáticos, levou-se em conta uma proporção de 30% dos novos casos (assim como faz Hill (2020), citando Bi et al. 2020; Mizumoto et al. 2020; e Nishiura et al. 2020). O 70% restante dos casos corresponde aos indivíduos que desenvolvem a forma moderada da doença, podendo progredir para os próximos estágios. **Ao contrário da implementação original**, aqui calculamos explicitamente o percentual de assintomáticos da população - ao invés de deixar essa tarefa a cargo de uma subrotina dentro do código.

In [4]:
place_id = "state_num_id"

In [5]:
place_specific_params = pd.read_csv("http://datasource.coronacidades.org/br/states/parameters")
place_specific_params = place_specific_params.merge(state_num_ids, on="state_num_id")
#place_specific_params.set_index(place_id, inplace=True)

# recalculate severity percentages considering the asymptomatic fraction
asymptomatic_proportion = config["br"]["seir_parameters"]["asymptomatic_proportion"]
place_specific_params["i0_percentage"] = asymptomatic_proportion
place_specific_params[["i1_percentage", "i2_percentage", "i3_percentage"]] = (
    place_specific_params[["i1_percentage", "i2_percentage", "i3_percentage"]]
    .apply(lambda x: x * ( 1 - asymptomatic_proportion))
)

place_specific_params

,fatality_ratio,hospitalized_by_age_perc,i1_percentage,i2_percentage,i3_percentage,state_num_id,data_last_refreshed,state_id,i0_percentage
0,0.004582,0.035272,0.675310,0.020433,0.004257,11,2021-01-31 04:09:07,RO,0.3
1,0.003882,0.029194,0.679565,0.016912,0.003523,12,2021-01-31 04:09:07,AC,0.3
2,0.003741,0.029431,0.679399,0.017049,0.003552,13,2021-01-31 04:09:07,AM,0.3
3,0.003333,0.029069,0.679652,0.016840,0.003508,14,2021-01-31 04:09:07,RR,0.3
4,0.004362,0.032281,0.677403,0.018701,0.003896,15,2021-01-31 04:09:07,PA,0.3
5,0.003405,0.028235,0.680235,0.016357,0.003408,16,2021-01-31 04:09:07,AP,0.3
6,0.005203,0.035553,0.675113,0.020596,0.004291,17,2021-01-31 04:09:07,TO,0.3
7,0.005125,0.033957,0.676230,0.019672,0.004098,21,2021-01-31 04:09:07,MA,0.3
8,0.006147,0.039047,0.672667,0.022620,0.004713,22,2021-01-31 04:09:07,PI,0.3
9,0.006366,0.039715,0.672200,0.023007,0.004793,23,2021-01-31 04:09:07,CE,0.3


### Obtém a proporção de casos nosocomiais no local

Um parâmetro fundamental para modelar uma epidemia é determinar a taxa de transmissão por indivíduo infectado e por unidade de tempo. Nos modelos original e modificado, essa taxa é denominada $\beta$ e subdividida em diferentes "*betas*" ($\beta_1$, $\beta_2$...), correspondentes a cada compartimento em que os indivíduos sejam transmissíveis.

Os valores assumidos por esses diferentes *betas* - que, inclusive, podem mudar no tempo, de acordo com as medidas de contenção adotadas - têm impacto direto na velocidade com que novos indivíduos suscetíveis serão expostos ao agente infeccioso. 

Na implementação original, esses *betas* eram calculados assumindo que a transmissão dos indivíduos infectados com sintomas moderados correspondia a 90% da transmissão efetiva, enquanto as taxas de transmissão dos indivíduos com sintomas graves e críticos eram iguais entre si e correspondiam, somadas, aos outros 10% da taxa de transmissão efetiva.

Na implementação alternativa, o cálculo deve incorporar novas taxas de transmissão, correspondentes aos indivíduos pré-sintomáticos transmissíveis ($\beta_E$) e assintomáticos ($\beta_0$). De maneira similar ao que sugere [Hill (2020)](https://alhill.shinyapps.io/COVID19seir/), assumimos que os casos pré-sintomáticos, assintomáticos e sintomáticos moderados têm a mesma transmissibilidade. Já os casos sintomáticos graves e críticos são calculados à parte, considerando que sejam os únicos responsáveis pela fração de casos cuja transmissão ocorre em hospitais (nosocomiais).

No Brasil, os casos de Síndrome Respiratória Aguda Grave - SRAG (que incluem os casos mais severos de COVID-19) são notificados por meio de um sistema de vigilância epidemiológico específico do Ministério da Saúde. Os dados estão disponíveis [publicamente][Dados SRAG] e incluem a informação de sobre o caso ter ou não origem nosocomial.

Utilizamos esse dado da vigilância da SRAG e extrapolamos para todo o conjunto das infecções por COVID-19, assumindo que a proporção de casos nosocomiais entre os casos de SRAG associáveis à COVID-19 seja identico à proporção de casos nosocomiais em outras formas mais leves da doença. A partir dessa proporção, inferimos os *betas* associados à transmissão hospitalar ($\beta_2$ e $\beta_3$) e, consequentemente, os demais *betas* correspondentes ao restante dos eventos de transmissão.

A aquisição dos dados e o cálculo completo está disponível no notebook [nosocomial_cases.ipynb], neste mesmo diretório. A seguir, apenas importamos a planilha local com os resultados por estado, incorporando-as aos parâmetros locais da doença.

[Dados SRAG]: https://opendatasus.saude.gov.br/dataset/bd-srag-2020
[nosocomial_cases.ipynb]: ./nosocomial_cases.ipynb

In [6]:
nosocomial = pd.read_csv("../data/nosocomial_srag.csv", usecols=["state_id", "nosocomial_proportion"])

place_specific_params = place_specific_params.merge(nosocomial, on="state_id").set_index("state_num_id")
place_specific_params

,fatality_ratio,hospitalized_by_age_perc,i1_percentage,i2_percentage,i3_percentage,data_last_refreshed,state_id,i0_percentage,nosocomial_proportion
state_num_id,,,,,,,,,
11,0.004582,0.035272,0.675310,0.020433,0.004257,2021-01-31 04:09:07,RO,0.3,0.067710
12,0.003882,0.029194,0.679565,0.016912,0.003523,2021-01-31 04:09:07,AC,0.3,0.000000
13,0.003741,0.029431,0.679399,0.017049,0.003552,2021-01-31 04:09:07,AM,0.3,0.020652
14,0.003333,0.029069,0.679652,0.016840,0.003508,2021-01-31 04:09:07,RR,0.3,0.014032
15,0.004362,0.032281,0.677403,0.018701,0.003896,2021-01-31 04:09:07,PA,0.3,0.015550
16,0.003405,0.028235,0.680235,0.016357,0.003408,2021-01-31 04:09:07,AP,0.3,0.008499
17,0.005203,0.035553,0.675113,0.020596,0.004291,2021-01-31 04:09:07,TO,0.3,0.097842
21,0.005125,0.033957,0.676230,0.019672,0.004098,2021-01-31 04:09:07,MA,0.3,0.007754
22,0.006147,0.039047,0.672667,0.022620,0.004713,2021-01-31 04:09:07,PI,0.3,0.032015


### Obtém os dados epidemiológicos de entrada

#### Importa dados históricos dos estados

**Ao contrário da implementação original**, utilizamos uma [versão customizada](https://github.com/bcbernardo/coronacidades-datasource) da ferramenta CoronaCidades, com uma [extremidade adicional](https://github.com/bcbernardo/coronacidades-datasource/blob/master/src/loader/endpoints/get_states_farolcovid_history.py) para registrar o histórico de evolução dos dados de:

- Casos e óbitos acumulados até cada data desde os primeiros casos nos estados
- Taxa de notificação e casos ativos estimados, tomando os 15 dias anteriores à data de referência
- Taxa de contágio (Rt), também tomando os 15 dias anteriores

Como a geração desses dados pode levar algumas horas, uma versão consolidada dos dados é disponibilizada localmente no repositório, no caminho `data/br-states-farolcovid-history.csv`, e na cópia do repositório no GitHub.

In [7]:
df_farol = pd.read_csv("../data/br-states-farolcovid-history.csv")

# add state ibge numbers
df_farol = df_farol.merge(state_num_ids, on="state_id")

# erase rows with missing data
df_farol = (
    df_farol
    .dropna()
    .reset_index()
)

df_farol["last_updated_cases"] = pd.to_datetime(df_farol["last_updated_cases"])

df_farol[["state_num_id", "state_id", "last_updated_cases", "confirmed_cases", "deaths", 
          "notification_rate", "active_cases", "rt_most_likely"]]

,state_num_id,state_id,last_updated_cases,confirmed_cases,deaths,notification_rate,active_cases,rt_most_likely
0,11,RO,2020-04-19,160.0,4.0,0.019748,7393.0,2.39
1,11,RO,2020-04-20,199.0,4.0,0.023810,7602.0,2.48
2,11,RO,2020-04-21,223.0,5.0,0.025545,7829.0,2.37
3,11,RO,2020-04-22,223.0,5.0,0.024465,8052.0,2.30
4,11,RO,2020-04-23,250.0,5.0,0.025484,8712.0,2.24
...,...,...,...,...,...,...,...,...
7616,53,DF,2021-01-15,262928.0,4406.0,0.373225,25191.0,1.01
7617,53,DF,2021-01-16,263756.0,4412.0,0.373225,25191.0,1.00
7618,53,DF,2021-01-17,264227.0,4421.0,0.373225,25191.0,1.01
7619,53,DF,2021-01-18,265274.0,4427.0,0.373225,25191.0,1.01


### Roda a simulação para cada registro histórico

**Ao contrário da implementação original**, preparamos e rodamos uma simulação para cada modelo (SEIR original e SEAPMDR) e para cada registro epidemiológico diário de cada UF desde o início dos casos. Isso permite avaliar o quanto cada um dos modelos acertou historicamente, comparando seu desempenho.

In [8]:
# create an empty DataFrame to receive predictions
df_predictions = pd.DataFrame()

for _, row in df_farol.iterrows():
    
    # prepare simulation parameters (S, E0, E1, I0, I1, I2, I3, R, D, Rt, N)
    params = prepare.prepare_simulation(row, place_id, config, place_specific_params)
    
    # run one simulation for each model available
    for model in ["SEAPMDR", "SEIR"]:
        
        # run simulation and get (best and worst) scenarios
        dfs_scenarios = simulator.run_simulation(params, config, model=model)
        
        # unpack scenarios
        for scenario in ["worst", "best"]:
            df = dfs_scenarios[scenario]
            df["scenario"] = scenario
            
            df["date_prediction"] = row["last_updated_cases"]
            df["state_num_id"] = row["state_num_id"]
            
            df= (
                df
                .reset_index()
                .rename(columns={"dias": "days"})
            )
            
            # add to DataFrame that collects all predictions
            df_predictions = pd.concat([df_predictions, df], ignore_index=True)

# show results
df_predictions

Rt: 4.69
Average infectious period: 8.182196853819898
Nosocomial prop.: 0.0677101054780537
Beta E, 0, 1: 0.47517954616790525
Beta 2, 3: 3.4733795567092636
--------------------------------------------
Rt: 2.39
Average infectious period: 8.182196853819898
Nosocomial prop.: 0.0677101054780537
Beta E, 0, 1: 0.22652178353773716
Beta 2, 3: 1.090794349654887
--------------------------------------------
Rt: 4.74
Average infectious period: 8.182196853819898
Nosocomial prop.: 0.0677101054780537
Beta E, 0, 1: 0.480751823910519
Beta 2, 3: 3.544104760026156
--------------------------------------------
Rt: 2.48
Average infectious period: 8.182196853819898
Nosocomial prop.: 0.0677101054780537
Beta E, 0, 1: 0.23587610145475632
Beta 2, 3: 1.155886381646209
--------------------------------------------
Rt: 4.48
Average infectious period: 8.182196853819898
Nosocomial prop.: 0.0677101054780537
Beta E, 0, 1: 0.4518344073989733
Beta 2, 3: 3.1862014552183258
--------------------------------------------
Rt: 2.

Rt: 2.48
Average infectious period: 8.182196853819898
Nosocomial prop.: 0.0677101054780537
Beta E, 0, 1: 0.23587610145475632
Beta 2, 3: 1.1558863816462088
--------------------------------------------
Rt: 1.78
Average infectious period: 8.182196853819898
Nosocomial prop.: 0.0677101054780537
Beta E, 0, 1: 0.16428994397126315
Beta 2, 3: 0.7004265885191887
--------------------------------------------
Rt: 2.43
Average infectious period: 8.182196853819898
Nosocomial prop.: 0.0677101054780537
Beta E, 0, 1: 0.23067426630139365
Beta 2, 3: 1.1194769948430536
--------------------------------------------
Rt: 1.79
Average infectious period: 8.182196853819898
Nosocomial prop.: 0.0677101054780537
Beta E, 0, 1: 0.16529201937271643
Beta 2, 3: 0.7061404973440163
--------------------------------------------
Rt: 2.4
Average infectious period: 8.182196853819898
Nosocomial prop.: 0.0677101054780537
Beta E, 0, 1: 0.22755914716023737
Beta 2, 3: 1.0979281121129099
--------------------------------------------
R

Rt: 1.5
Average infectious period: 8.182196853819898
Nosocomial prop.: 0.0677101054780537
Beta E, 0, 1: 0.13651046245079457
Beta 2, 3: 0.5492380963261472
--------------------------------------------
Rt: 1.81
Average infectious period: 8.182196853819898
Nosocomial prop.: 0.0677101054780537
Beta E, 0, 1: 0.1672981362896091
Beta 2, 3: 0.7176345458973751
--------------------------------------------
Rt: 1.52
Average infectious period: 8.182196853819898
Nosocomial prop.: 0.0677101054780537
Beta E, 0, 1: 0.13847613955977772
Beta 2, 3: 0.559482613987525
--------------------------------------------
Rt: 1.79
Average infectious period: 8.182196853819898
Nosocomial prop.: 0.0677101054780537
Beta E, 0, 1: 0.16529201937271634
Beta 2, 3: 0.7061404973440164
--------------------------------------------
Rt: 1.52
Average infectious period: 8.182196853819898
Nosocomial prop.: 0.0677101054780537
Beta E, 0, 1: 0.13847613955977772
Beta 2, 3: 0.559482613987525
--------------------------------------------
Rt: 

Rt: 1.4
Average infectious period: 8.182196853819898
Nosocomial prop.: 0.0677101054780537
Beta E, 0, 1: 0.12672743078016555
Beta 2, 3: 0.4992668839437127
--------------------------------------------
Rt: 1.26
Average infectious period: 8.182196853819898
Nosocomial prop.: 0.0677101054780537
Beta E, 0, 1: 0.11316332011998387
Beta 2, 3: 0.43275736327594994
--------------------------------------------
Rt: 1.39
Average infectious period: 8.182196853819898
Nosocomial prop.: 0.0677101054780537
Beta E, 0, 1: 0.1257533676792274
Beta 2, 3: 0.49438358895610274
--------------------------------------------
Rt: 1.25
Average infectious period: 8.182196853819898
Nosocomial prop.: 0.0677101054780537
Beta E, 0, 1: 0.11220058330176133
Beta 2, 3: 0.4281584297579773
--------------------------------------------
Rt: 1.39
Average infectious period: 8.182196853819898
Nosocomial prop.: 0.0677101054780537
Beta E, 0, 1: 0.12575336767922743
Beta 2, 3: 0.49438358895610285
--------------------------------------------

Rt: 1.18
Average infectious period: 8.182196853819898
Nosocomial prop.: 0.0677101054780537
Beta E, 0, 1: 0.1054850932895259
Beta 2, 3: 0.3965250233703493
--------------------------------------------
Rt: 1.07
Average infectious period: 8.182196853819898
Nosocomial prop.: 0.0677101054780537
Beta E, 0, 1: 0.09501839027179776
Beta 2, 3: 0.3487700279219565
--------------------------------------------
Rt: 1.18
Average infectious period: 8.182196853819898
Nosocomial prop.: 0.0677101054780537
Beta E, 0, 1: 0.10548509328952592
Beta 2, 3: 0.3965250233703493
--------------------------------------------
Rt: 1.07
Average infectious period: 8.182196853819898
Nosocomial prop.: 0.0677101054780537
Beta E, 0, 1: 0.09501839027179775
Beta 2, 3: 0.34877002792195644
--------------------------------------------
Rt: 1.17
Average infectious period: 8.182196853819898
Nosocomial prop.: 0.0677101054780537
Beta E, 0, 1: 0.10452916634108178
Beta 2, 3: 0.3920854127550137
--------------------------------------------


...

Rt: 1.01
Average infectious period: 8.20020395645838
Nosocomial prop.: 0.0408694192682108
Beta E, 0, 1: 0.09188174313456055
Beta 2, 3: 0.17719766949728366
--------------------------------------------
Rt: 0.91
Average infectious period: 8.20020395645838
Nosocomial prop.: 0.0408694192682108
Beta E, 0, 1: 0.08225929341770365
Beta 2, 3: 0.15519407404857744
--------------------------------------------
Rt: 0.99
Average infectious period: 8.20020395645838
Nosocomial prop.: 0.0408694192682108
Beta E, 0, 1: 0.08994946210486261
Beta 2, 3: 0.17271380701391892
--------------------------------------------
Rt: 0.9
Average infectious period: 8.20020395645838
Nosocomial prop.: 0.0408694192682108
Beta E, 0, 1: 0.08130248270536697
Beta 2, 3: 0.153050568611451
--------------------------------------------
Rt: 0.98
Average infectious period: 8.20020395645838
Nosocomial prop.: 0.0408694192682108
Beta E, 0, 1: 0.08898476609597679
Beta 2, 3: 0.17048753048448415
--------------------------------------------
Rt:

Rt: 1.04
Average infectious period: 8.20020395645838
Nosocomial prop.: 0.0408694192682108
Beta E, 0, 1: 0.09478731861054646
Beta 2, 3: 0.18400201822030426
--------------------------------------------
Rt: 0.94
Average infectious period: 8.20020395645838
Nosocomial prop.: 0.0408694192682108
Beta E, 0, 1: 0.08513572042417311
Beta 2, 3: 0.16168635308325055
--------------------------------------------
Rt: 1.05
Average infectious period: 8.20020395645838
Nosocomial prop.: 0.0408694192682108
Beta E, 0, 1: 0.0957577333820414
Beta 2, 3: 0.18629115772231084
--------------------------------------------
Rt: 0.95
Average infectious period: 8.20020395645838
Nosocomial prop.: 0.0408694192682108
Beta E, 0, 1: 0.08609651171844594
Beta 2, 3: 0.16387109541003947
--------------------------------------------
Rt: 1.06
Average infectious period: 8.20020395645838
Nosocomial prop.: 0.0408694192682108
Beta E, 0, 1: 0.0967290854958715
Beta 2, 3: 0.18859084030223314
--------------------------------------------
Rt

Rt: 1.12
Average infectious period: 8.20020395645838
Nosocomial prop.: 0.0408694192682108
Beta E, 0, 1: 0.1025766356408945
Beta 2, 3: 0.2026114017392857
--------------------------------------------
Rt: 1.02
Average infectious period: 8.20020395645838
Nosocomial prop.: 0.0408694192682108
Beta E, 0, 1: 0.09284931898156022
Beta 2, 3: 0.1794552929458901
--------------------------------------------
Rt: 1.13
Average infectious period: 8.20020395645838
Nosocomial prop.: 0.0408694192682108
Beta E, 0, 1: 0.10355442633228625
Beta 2, 3: 0.20498541716687874
--------------------------------------------
Rt: 1.03
Average infectious period: 8.20020395645838
Nosocomial prop.: 0.0408694192682108
Beta E, 0, 1: 0.09381784564645691
Beta 2, 3: 0.18172340291258424
--------------------------------------------
Rt: 1.13
Average infectious period: 8.20020395645838
Nosocomial prop.: 0.0408694192682108
Beta E, 0, 1: 0.10355442633228623
Beta 2, 3: 0.20498541716687876
--------------------------------------------
Rt:

Rt: 1.1
Average infectious period: 8.20020395645838
Nosocomial prop.: 0.0408694192682108
Beta E, 0, 1: 0.10062377899722551
Beta 2, 3: 0.1978953801391432
--------------------------------------------
Rt: 1.0
Average infectious period: 8.20020395645838
Nosocomial prop.: 0.0408694192682108
Beta E, 0, 1: 0.09091512265128032
Beta 2, 3: 0.17495051377410356
--------------------------------------------
Rt: 1.09
Average infectious period: 8.20020395645838
Nosocomial prop.: 0.0408694192682108
Beta E, 0, 1: 0.09964872163729206
Beta 2, 3: 0.19555333580228212
--------------------------------------------
Rt: 0.99
Average infectious period: 8.20020395645838
Nosocomial prop.: 0.0408694192682108
Beta E, 0, 1: 0.08994946210486263
Beta 2, 3: 0.17271380701391892
--------------------------------------------
Rt: 1.1
Average infectious period: 8.20020395645838
Nosocomial prop.: 0.0408694192682108
Beta E, 0, 1: 0.10062377899722551
Beta 2, 3: 0.1978953801391432
--------------------------------------------
Rt: 1

,days,S,E0,E1,I0,I1,I2,I3,R,D,N,E,scenario,model,date_prediction,state_num_id
0,1,1.732049e+06,25912.806364,11160.746304,2217.900000,4992.565145,151.063328,31.471527,705.000000,4.000000,1777225.0,37073.552668,worst,SEAPMDR,2020-04-19,11
1,2,1.721016e+06,28030.924722,13808.962454,3614.379355,8279.039409,157.857568,31.914058,2278.094257,8.262260,1777225.0,41839.887176,worst,SEAPMDR,2020-04-19,11
2,3,1.706539e+06,32478.771646,16316.011132,5142.710324,11868.865484,179.164783,32.690375,4654.920221,12.602302,1777225.0,48794.782778,worst,SEAPMDR,2020-04-19,11
3,4,1.688375e+06,38810.685859,19276.555747,6810.440671,15780.315246,214.091163,34.140381,7906.630186,17.088667,1777225.0,58087.241606,worst,SEAPMDR,2020-04-19,11
4,5,1.666062e+06,46889.011036,22981.920854,8681.686565,20163.552132,262.315575,36.546580,12126.343299,21.831493,1777225.0,69870.931890,worst,SEAPMDR,2020-04-19,11
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2774039,87,2.211629e+06,NaN,NaN,NaN,2270.478843,864.849935,244.496192,780147.817428,10929.996403,3007710.7,1624.007466,best,SEIR,2021-01-19,53
2774040,88,2.211346e+06,NaN,NaN,NaN,2217.484408,844.731456,238.850212,780513.170436,10964.387711,3007710.7,1585.994636,best,SEIR,2021-01-19,53
2774041,89,2.211070e+06,NaN,NaN,NaN,2165.706149,825.071624,233.330554,780870.002262,10997.984553,3007710.7,1548.859116,best,SEIR,2021-01-19,53
2774042,90,2.210800e+06,NaN,NaN,NaN,2115.117114,805.860408,227.934625,781218.508181,11030.804718,3007710.7,1512.581241,best,SEIR,2021-01-19,53


In [9]:
df_predictions.to_csv("../data/br-states-simulacovid-predictions.csv", index=False)